In [3]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [4]:

# Step 1: Load the Dataset
file_path = r"C:\Users\ujair shaha\Desktop\Projects\Product Recommendation System\models\final_walmart_product_data.csv"
df = pd.read_csv(file_path)

In [5]:
df.head()

,user_id,ProdID,Category,Brand,Name,ImageURL,Description,Tags,Rating
0,USER_161,43844b4d01b62ddf5b329af60afd243a,Personal Care,Crest,Crest Pls Sc Outlst Ex Wh Size 4z Crest Extra ...,https://i5.walmartimages.com/asr/8604b82d-4c15...,DetailsCrest Extra White Plus Outlast Toothpas...,Crest Pls Sc Outlst Ex Wh Size 4z Crest Extra ...,4
1,USER_42,56182586933bbdea8c572b0af1af513b,Premium Beauty,Kerastase,"Kerastase Discipline Maskeratine, 500ml",https://i5.walmartimages.com/asr/0e200420-2b90...,NaN,"Kerastase Discipline Maskeratine, 500ml, Wal-m...",3
2,USER_1335,446f1f57405d2df74e8d89a8dbd458ba,Health,Vega,"Vega Protein Made Simple, Vanilla, 10 Servings...",https://i5.walmartimages.com/asr/0766d3dd-737c...,Whatâ€™s on the inside matters. Thatâ€™s why V...,"Vega Protein Made Simple, Vanilla, 10 Servings...",3
3,USER_34,658e17ace9dd43ba0541c590434eb34e,Premium Beauty,Mac,"MAC Mineralize Concealer, NW35",https://i5.walmartimages.com/asr/cffd0ed5-1d66...,NaN,"MAC Mineralize Concealer, NW35, Wal-mart, Walm...",1
4,USER_422,ca95004fec7e177cbbb8c735db5d4a82,Home,Prevail,"Prevail Premium Quilted Adult Washcloths, 48 C...",https://i5.walmartimages.com/asr/f3c2a1c5-042d...,NaN,"Prevail Premium Quilted Adult Washcloths, 48 C...",3


In [6]:
df['user_id'].nunique()

1378

# Collabrative filtering.

## User-Item based filtering

In [7]:
# Recommends items that are frequently interacted with together.
# Example: "People who bought this also bought X."

In [8]:
user_item_matrix = df.pivot_table(index='user_id', columns='ProdID', values='Rating', fill_value=0)

In [9]:

# Step 3: Apply SVD for Collaborative Filtering
svd = TruncatedSVD(n_components=50)
user_factors = svd.fit_transform(user_item_matrix)
item_factors = svd.components_.T

In [10]:
def item_based_recommendation(user_id, user_item_matrix, user_factors, item_factors, top_n=15):
    user_idx = user_item_matrix.index.get_loc(user_id)
    user_vector = user_factors[user_idx]
    
    # Calculate predicted ratings
    predicted_ratings = user_vector.dot(item_factors.T)
    
    # Exclude already rated items
    user_rated_items = user_item_matrix.loc[user_id] > 0
    predicted_ratings[user_rated_items.values] = -np.inf  # Assign -inf to already rated items
    
    # Get indices of the top N recommended items
    unrated_item_indices = np.argsort(predicted_ratings)[::-1]
    unrated_item_indices = unrated_item_indices[:top_n]  # Ensure we don't exceed available items
    
    # Get corresponding product IDs
    recommended_item_ids = user_item_matrix.columns[unrated_item_indices]
    
    # Fetch product details for the recommendations
    recommended_products = (
        df[df['ProdID'].isin(recommended_item_ids)]
        [['ProdID', 'Name','Brand', 'Category','ImageURL']]
        .drop_duplicates(subset='ProdID')
        .head(top_n)
    )
    
    return recommended_products


In [11]:
# Step 5: Recommend for a Specific User
user_id = "USER_1335"  # Example user
recommendations = item_based_recommendation(user_id, user_item_matrix, user_factors, item_factors)


In [12]:
recommendations

,ProdID,Name,Brand,Category,ImageURL
9,3fdbd7a1c4cf784e11363e6eb206d633,"Sally Hansen Miracle Gel Nail Polish, Terra-Co...",Sally Hansen,Beauty,https://i5.walmartimages.com/asr/acb03bbb-fa57...
51,290366d729851491f1b36b0f1141487e,Rusk Sensories Pure Mandarin and Jasmine Shamp...,Rusk,Premium Beauty,https://i5.walmartimages.com/asr/736ac3be-6c68...
166,613b3fd96d7b5d179ab9951438c402c2,PALM KERNEL OIL ORGANIC CARRIER COLD PRESSED S...,H&B Oils Center,Health,https://i5.walmartimages.com/asr/cd51485b-dc2e...
482,40b98386b64701d2e9c9c036e2c92fb4,LAC High Shine Lipgloss Dynamite,L.A. Colors,Beauty,https://i5.walmartimages.com/asr/d0e2c54b-59e4...
515,a7b360b8a899d0fdf513cb0e8e41c75c,OLAY Total Effects 7-In-1 Anti-Aging Daily Moi...,Olay,Shop by Brand,https://i5.walmartimages.com/asr/d7477175-4280...
573,3036ca55265f15bebc1e01a6692977cb,Blue 3 PCS/Set Tile Grout Power Scrubber Clean...,iMounTEK,Household Essentials,https://i5.walmartimages.com/asr/8a3553de-acda...
574,4b817bec74f6af2feb0f39fa3335690d,Waterpik Cordless Express Portable Water Floss...,Waterpik,Personal Care,https://i5.walmartimages.com/asr/35b9aff3-5a5f...
856,0007704300095379400000002000000000067661936US,"St. Ives Acne Control With Salicylic Acid, Non...",St. Ives,Personal Care,https://i5.walmartimages.com/asr/c3236124-7c83...
1285,3b47e5cfacef18f26d07ef7c94a9607e,Plant Therapy Lemon Organic Essential Oil | 10...,Plant Therapy,Health,https://i5.walmartimages.com/asr/06d4f1a2-de0e...
1665,acf1ffbecdb91dda8b970d7d8a330f53,Wood Nymph Eye Colour,Larenim,Premium Beauty,https://i5.walmartimages.com/asr/b4dc2291-c82e...


## User-User Based recommendation

In [13]:
# Recommends items liked by similar users
# Example: "People like you also bought X."

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
# Calculate cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)


In [16]:
user_similarity_df.head()

user_id,USER_10,USER_100,USER_1000,USER_1001,USER_1002,USER_1003,USER_1004,USER_1005,USER_1006,USER_1007,...,USER_99,USER_990,USER_991,USER_992,USER_993,USER_994,USER_995,USER_996,USER_997,USER_998
user_id,,,,,,,,,,,,,,,,,,,,,
USER_10,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
USER_100,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
USER_1000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
USER_1001,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
USER_1002,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.342368,0.0,0.0,0.0,0.0,0.0


In [17]:
def user_based_recommendation(user_id, user_item_matrix, user_similarity_df, top_n=10):
    # Get the list of users that are most similar to the target user
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:].head(top_n)
    
    # Get ratings of similar users
    similar_users_ratings = user_item_matrix.loc[similar_users.index]
    
    # Predict ratings for the target user by averaging the ratings from similar users
    predicted_ratings = similar_users_ratings.T.dot(similar_users.values) / similar_users.values.sum()
    
    # Get the items the user has not rated yet
    user_rated_items = user_item_matrix.loc[user_id] > 0
    unrated_items = predicted_ratings[user_rated_items == False]
    
    # Sort unrated items by predicted rating
    recommended_items = unrated_items.sort_values(ascending=False).head(top_n)
    
    # Fetch product details for the recommended items
    recommended_product_details = df[df['ProdID'].isin(recommended_items.index)][['ProdID', 'Name','Brand', 'Category','ImageURL']].drop_duplicates(subset='ProdID')
    
    return recommended_product_details


In [18]:
# Example of recommending products for a user
user_id = 'USER_10'  # Example user
recommended_products = user_based_recommendation(user_id, user_item_matrix, user_similarity_df, top_n=10)

In [19]:
recommended_products

,ProdID,Name,Brand,Category,ImageURL
9,3fdbd7a1c4cf784e11363e6eb206d633,"Sally Hansen Miracle Gel Nail Polish, Terra-Co...",Sally Hansen,Beauty,https://i5.walmartimages.com/asr/acb03bbb-fa57...
38,2d8c1d2a6e81a1510224d041b782c391,1 Beauty Skin Cloth Exfoliating Nylon Shower B...,KIRWV KILWFXGH,Personal Care,https://i5.walmartimages.com/asr/aa9b8dca-ed07...
72,4f13cc7607526a1abf1b158c59ac3d0e,"Abena Light Premium Incontinence Pad, Normal 2...",Abena,Personal Care,https://i5.walmartimages.com/asr/bcc20471-33fb...
239,978c415f19c89e3a48527196aae23d29,6 Pack - AirWick FRESHMATIC Ultra - Refill Pur...,Air Wick,Household Essentials,https://i5.walmartimages.com/asr/e4072d8e-bdd3...
327,5c127021631153ab2d4e6af81532c30a,"Boulder Clean Granite and Stainless Cleaner, L...",Boulder Clean,Household Essentials,https://i5.walmartimages.com/asr/d0f5572e-ace2...
581,7bfb9831d48bad6e15edcca44c7a9b85,Reviva Glycolic Acid Facial Cleanser For Matur...,Reviva Labs,Beauty,https://i5.walmartimages.com/asr/5bef8491-d0bc...
772,1eace202693bf1654afee05366cb79d4,"Reynolds Kitchens Premium Slow Cooker Liners, ...",Reynolds Wrap,Household Essentials,https://i5.walmartimages.com/asr/2cb73ae5-e431...
1287,b46684504d17799e61b6571b6b44906d,2 Pc Scissor Style Eyebrow Tweezers Set Hair R...,AllTopBargains,Beauty,https://i5.walmartimages.com/asr/1d3b5beb-afb0...
1555,8bab15aded2de70f367813ef16938178,"Navitas Organics Maca Gelatinized Powder, 8 oz...",Navitas Organics,Health,https://i5.walmartimages.com/asr/271dc9a9-4678...
2512,6bd24f066f8171a1ecca97515a1ce6ae,NutriCology ProGreens 30 Day Supply 9.27 oz (2...,Nutricology,Health,https://i5.walmartimages.com/asr/df85e1ca-f43d...


# Content-Based Filtering:

In [20]:
# Recommends items similar to what the user has interacted with, based on features like:
# Keywords in product descriptions.
# Categories, tags, or specifications.
# Example: "You liked X, so you might also like Y (with similar features)."

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
df.head()

,user_id,ProdID,Category,Brand,Name,ImageURL,Description,Tags,Rating
0,USER_161,43844b4d01b62ddf5b329af60afd243a,Personal Care,Crest,Crest Pls Sc Outlst Ex Wh Size 4z Crest Extra ...,https://i5.walmartimages.com/asr/8604b82d-4c15...,DetailsCrest Extra White Plus Outlast Toothpas...,Crest Pls Sc Outlst Ex Wh Size 4z Crest Extra ...,4
1,USER_42,56182586933bbdea8c572b0af1af513b,Premium Beauty,Kerastase,"Kerastase Discipline Maskeratine, 500ml",https://i5.walmartimages.com/asr/0e200420-2b90...,NaN,"Kerastase Discipline Maskeratine, 500ml, Wal-m...",3
2,USER_1335,446f1f57405d2df74e8d89a8dbd458ba,Health,Vega,"Vega Protein Made Simple, Vanilla, 10 Servings...",https://i5.walmartimages.com/asr/0766d3dd-737c...,Whatâ€™s on the inside matters. Thatâ€™s why V...,"Vega Protein Made Simple, Vanilla, 10 Servings...",3
3,USER_34,658e17ace9dd43ba0541c590434eb34e,Premium Beauty,Mac,"MAC Mineralize Concealer, NW35",https://i5.walmartimages.com/asr/cffd0ed5-1d66...,NaN,"MAC Mineralize Concealer, NW35, Wal-mart, Walm...",1
4,USER_422,ca95004fec7e177cbbb8c735db5d4a82,Home,Prevail,"Prevail Premium Quilted Adult Washcloths, 48 C...",https://i5.walmartimages.com/asr/f3c2a1c5-042d...,NaN,"Prevail Premium Quilted Adult Washcloths, 48 C...",3


In [23]:
# Step 2: Use TF-IDF vectorizer to convert text into numerical features
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Tags'])

In [24]:
# Step 3: Compute the cosine similarity between products
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [25]:
# Step 4: Define a function to recommend top N similar items based on a given product_id
def recommend_content_based(product_id, df, cosine_sim, top_n=15):
    # Get the index of the product
    idx = df[df['ProdID'] == product_id].index[0]
    
    # Get the similarity scores for the product with all other products
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the products based on similarity scores in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the top N most similar products (excluding the product itself)
    sim_scores = sim_scores[1:top_n+1]
    
    # Get the product indices and corresponding similarity scores
    product_indices = [i[0] for i in sim_scores]
    
    # Fetch the product details for the recommended products
    recommended_products = df.iloc[product_indices][['ProdID', 'Name','Brand', 'Category','ImageURL']].drop_duplicates(subset='ProdID')
    
    return recommended_products



In [26]:

# Example usage
product_id = '43844b4d01b62ddf5b329af60afd243a'  # Replace with the product_id you want to get recommendations for
recommended_products = recommend_content_based(product_id, df, cosine_sim)

In [27]:
recommended_products

,ProdID,Name,Brand,Category,ImageURL
780,43844b4d01b62ddf5b329af60afd243a,Crest Pls Sc Outlst Ex Wh Size 4z Crest Extra ...,Crest,Personal Care,https://i5.walmartimages.com/asr/8604b82d-4c15...
2337,55363dedad842c4b9da5524f58484f7d,Crest 4.8 Oz 3D White Toothpaste,Crest,Personal Care,https://i5.walmartimages.com/asr/c9f52c1f-c2ce...
559,4704dc7c82d15b1fed9111860ead205b,"Crest Complete Whitening + Scope Toothpaste, 5...",Crest,Personal Care,https://i5.walmartimages.com/asr/cb17ccbd-526a...
3026,9c38d1bc68e91ccbcb9225fccdb9acca,"Crest + Scope Complete Whitening Toothpaste, M...",Crest,Personal Care,https://i5.walmartimages.com/asr/ec72a0a8-8322...
634,cf6a52ab8251c299898aa10893446441,3 Pack - Crest Cavity Protection Toothpaste Ge...,Crest,Personal Care,https://i5.walmartimages.com/asr/a4a16d9d-d88d...
859,f45e0575ce4bc36cc17167bc0e7a8ff3,"Crest Charcoal 3D White Toothpaste, Whitening ...",Crest,Personal Care,https://i5.walmartimages.com/asr/c333844f-5d63...


## Search Query based Recommendation

In [28]:
# Function to search products based on a query
def search_products(query, top_n=10):
    # Transform the query using the same TF-IDF vectorizer
    query_tfidf = tfidf_vectorizer.transform([query])
    
    # Compute cosine similarity between the query and product descriptions
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    
    # Get indices of the most similar products
    similar_product_indices = cosine_similarities.argsort()[:-top_n-1:-1]
    
    # Get the top recommended products
    recommended_products = df.iloc[similar_product_indices]
    
    return recommended_products[['ProdID', 'Name','Brand', 'Category','ImageURL']].drop_duplicates(subset='ProdID')


In [29]:
# Example search query
query = 'Toothpaste for teeth whitening'
recommended = search_products(query)

In [30]:
recommended

,ProdID,Name,Brand,Category,ImageURL
211,99f668de9d0d9967784da0ea45843b7f,"Colgate Total Whitening Toothpaste, Advanced W...",Colgate,Seasonal,https://i5.walmartimages.com/asr/4822fab2-f856...
1607,ad4c9099fa82aaf0de5b743186696122,"Colgate Total Whitening Toothpaste (6.3 Ounce,...",Colgate,Personal Care,https://i5.walmartimages.com/asr/9cb2e802-10d1...
559,4704dc7c82d15b1fed9111860ead205b,"Crest Complete Whitening + Scope Toothpaste, 5...",Crest,Personal Care,https://i5.walmartimages.com/asr/cb17ccbd-526a...
1305,5464dbe6991a9cb5693b2202bec5d496,Sensodyne Sensitivity & Gum Whitening Toothpas...,Sensodyne,Personal Care,https://i5.walmartimages.com/asr/dcbad31a-39e4...


# Hybrid Recommendation System (Collaborative + Content-Based)

In [31]:
def hybrid_recommend(user_id, user_item_matrix, user_factors, item_factors, cosine_sim, df, top_n=10):
    # User index and vector
    user_idx = user_item_matrix.index.get_loc(user_id)
    user_vector = user_factors[user_idx]
    
    # Collaborative predictions
    collaborative_scores = user_vector.dot(item_factors.T)
    
    # Content similarity scores
    already_rated = user_item_matrix.loc[user_id] > 0
    collaborative_scores[already_rated.values] = -np.inf  # Exclude already rated items
    
    # Blend Collaborative and Content-Based Scores
    blended_scores = np.zeros(collaborative_scores.shape)
    for i, score in enumerate(collaborative_scores):
        if score != -np.inf:
            blended_scores[i] = score + cosine_sim[i].mean()  # Weight collaborative + content
    
    # Get top N recommendations
    top_indices = np.argsort(blended_scores)[::-1][:top_n]
    recommended_item_ids = user_item_matrix.columns[top_indices]
    
    # Get product details
    recommended_products = (
        df[df['ProdID'].isin(recommended_item_ids)]
        [['ProdID', 'Name','Brand', 'Category','ImageURL']]
        .drop_duplicates(subset='ProdID')
    )
    
    return recommended_products



In [32]:
# Example Usage
user_id = "USER_10"  # Replace with an actual user ID
top_recommendations = hybrid_recommend(user_id, user_item_matrix, user_factors, item_factors, cosine_sim, df)

In [33]:
top_recommendations

,ProdID,Name,Brand,Category,ImageURL
72,4f13cc7607526a1abf1b158c59ac3d0e,"Abena Light Premium Incontinence Pad, Normal 2...",Abena,Personal Care,https://i5.walmartimages.com/asr/bcc20471-33fb...
169,6b31ac21d75ea9504a7736389bd8329c,"COVERGIRL Clean Matte BB Cream, 550 Medium Dee...",COVERGIRL,Beauty,https://i5.walmartimages.com/asr/db335350-94c9...
239,978c415f19c89e3a48527196aae23d29,6 Pack - AirWick FRESHMATIC Ultra - Refill Pur...,Air Wick,Household Essentials,https://i5.walmartimages.com/asr/e4072d8e-bdd3...
315,9ae831a4b2f76277eaf11eb10c5e2252,Rimmel London Exaggerate Smoke N Shine Eyeline...,Rimmel,Beauty,https://i5.walmartimages.com/asr/b269f9e2-5cf7...
327,5c127021631153ab2d4e6af81532c30a,"Boulder Clean Granite and Stainless Cleaner, L...",Boulder Clean,Household Essentials,https://i5.walmartimages.com/asr/d0f5572e-ace2...
581,7bfb9831d48bad6e15edcca44c7a9b85,Reviva Glycolic Acid Facial Cleanser For Matur...,Reviva Labs,Beauty,https://i5.walmartimages.com/asr/5bef8491-d0bc...
721,7c10fdac493fe0d5f932cbede556279c,Dr. Woods Shea Vision Pure Almond Castile Soap...,Dr. Woods,Personal Care,https://i5.walmartimages.com/asr/2cf7aa4c-48dc...
1413,3fb3798a0a33adb5de4097170f127b59,"e.l.f. Day to Night Lipstick Duo, Need It Nudes",e.l.f. Cosmetics,Beauty,https://i5.walmartimages.com/asr/9c17d9b0-35da...
1673,de74bbf2e9fe2ef3b5dfdc37a8275f79,"Egyptian Magic All Purpose Skin Cream, 0.25 OZ",Egyptian Magic,Personal Care,https://i5.walmartimages.com/asr/80fece34-e386...
2640,06a8b3a49f2add3c8b32e7ca5a432fe0,Karma Organic Non-Allergenic Nail Polish Vegan...,Karma Organic,Premium Beauty,https://i5.walmartimages.com/asr/a944277d-15d8...


# Popular item based Recommendation

In [34]:
popularity_data = df.groupby('ProdID').agg({
    'Rating': ['mean', 'count'],
    'Name': 'first',
    'Category': 'first',
    'Brand': 'first',
    'ImageURL': 'first'
}).reset_index()

In [35]:
popularity_data.head()

ProdID    Rating        \
                                                      mean count   
0  0001111104242179400000002000000000067846570US  1.833333    30   
1  0005508600654579400000002000000000067913543US  2.636364    11   
2  0007704300095379400000002000000000067661936US  2.142857     7   
3  0007940026654579400000002000000000067239436US  2.250000     4   
4  0007940046105679400000002000000000067805363US  2.200000    15   

                                                Name        Category  \
                                               first           first   
0  Dove Refresh+Care Detox & Purify Dry Shampoo, ...         Beauty    
1  ($11 Value) Love Beauty and Planet Coconut Wat...  Personal Care    
2  St. Ives Acne Control With Salicylic Acid, Non...  Personal Care    
3  Dove Men+Care Extra Fresh Deodorant Stick, 3 o...       Seasonal    
4                        AXE Phoenix Body Wash 32 oz  Personal Care    

                    Brand                                           ImageURL  
                    first                                              first  
0                    Dove  https://i5.walmartimages.com/asr/c1eb8030-c57f...  
1  Love Beauty and Planet  https://i5.walmartimages.com/asr/0348d82d-2a13...  
2                St. Ives  https://i5.walmartimages.com/asr/c3236124-7c83...  
3           Dove Men+Care  https://i5.walmartimages.com/asr/b5e8998a-b91a...  
4                     Axe  https://i5.walmartimages.com/asr/bfcb35fc-78ea...

In [36]:
popularity_data.columns = ['product_id', 'avg_rating', 'rating_count', 'product_name', 'product_category','product_brand','ImageURl']


In [37]:
# Step 2: Sort by Popularity
# Higher average rating and higher number of ratings are considered popular
popularity_data['popularity_score'] = popularity_data['avg_rating'] * popularity_data['rating_count']
popular_products = popularity_data.sort_values(by='popularity_score', ascending=False)

In [38]:
# Step 3: Get Top N Recommendations
def recommend_popular_items(top_n=12):
    return popular_products.head(top_n)[['product_id', 'product_name', 'product_category','product_brand','ImageURl', 'avg_rating', 'rating_count']]


In [39]:
top_recommendations = recommend_popular_items(top_n=12)


In [42]:
top_recommendations.head()

,product_id,product_name,product_category,product_brand,ImageURl,avg_rating,rating_count
158,1007940052235879400000002000000000067565785US,Suave Extra Hold 7 Shaping Mousse 9 oz (Pack o...,Beauty,Suave,https://i5.walmartimages.com/asr/05f5ba16-335d...,2.457143,35
0,0001111104242179400000002000000000067846570US,"Dove Refresh+Care Detox & Purify Dry Shampoo, ...",Beauty,Dove,https://i5.walmartimages.com/asr/c1eb8030-c57f...,1.833333,30
162,1030521004419479400000002000000000067944214US,"Vaseline Petroleum Jelly, Original 3.75 oz",Personal Care,Vaseline,https://i5.walmartimages.com/asr/fa508c93-0811...,2.333333,18
4,0007940046105679400000002000000000067805363US,AXE Phoenix Body Wash 32 oz,Personal Care,Axe,https://i5.walmartimages.com/asr/bfcb35fc-78ea...,2.200000,15
1312,982217d808e110f98c97bd79dfee196b,COVERGIRL Outlast All-Day Moisturizing Lip Col...,Beauty,COVERGIRL,https://i5.walmartimages.com/asr/45cf8411-9119...,2.727273,11


In [41]:
# top_recommendations.to_csv('Top_Rated.csv',index=False)

In [51]:
ProdID = df['ProdID'].unique().tolist()

In [52]:
user_id = df['user_id'].unique().tolist()

In [50]:
unique_user_id

['USER_161',
 'USER_42',
 'USER_1335',
 'USER_34',
 'USER_422',
 'USER_1242',
 'USER_212',
 'USER_902',
 'USER_680',
 'USER_814',
 'USER_970',
 'USER_1170',
 'USER_588',
 'USER_430',
 'USER_560',
 'USER_13',
 'USER_54',
 'USER_615',
 'USER_1042',
 'USER_504',
 'USER_741',
 'USER_17',
 'USER_222',
 'USER_44',
 'USER_604',
 'USER_234',
 'USER_229',
 'USER_541',
 'USER_897',
 'USER_1308',
 'USER_333',
 'USER_389',
 'USER_960',
 'USER_965',
 'USER_530',
 'USER_215',
 'USER_836',
 'USER_163',
 'USER_546',
 'USER_1054',
 'USER_164',
 'USER_284',
 'USER_1108',
 'USER_1116',
 'USER_1086',
 'USER_353',
 'USER_573',
 'USER_937',
 'USER_118',
 'USER_701',
 'USER_1400',
 'USER_657',
 'USER_1318',
 'USER_626',
 'USER_1002',
 'USER_520',
 'USER_1150',
 'USER_16',
 'USER_1013',
 'USER_1241',
 'USER_1165',
 'USER_1133',
 'USER_21',
 'USER_802',
 'USER_1149',
 'USER_957',
 'USER_767',
 'USER_636',
 'USER_577',
 'USER_3',
 'USER_360',
 'USER_181',
 'USER_1126',
 'USER_725',
 'USER_8',
 'USER_1017',
 'US

In [48]:
unique_product_id

['43844b4d01b62ddf5b329af60afd243a',
 '56182586933bbdea8c572b0af1af513b',
 '446f1f57405d2df74e8d89a8dbd458ba',
 '658e17ace9dd43ba0541c590434eb34e',
 'ca95004fec7e177cbbb8c735db5d4a82',
 'dba06a20c3b572560a96a7d71779dcb6',
 '0073291322733479400000002000000000067662080US',
 '561eefa66befd5d2f53e750d9e104bca',
 '3fd3710d5934260afed0c7bb7e3e2dde',
 '3fdbd7a1c4cf784e11363e6eb206d633',
 'f09ec8d1330bd19adc34484ed7647172',
 '0c1d7dacc18798de15a0af021311a7f7',
 '609cf0cb732e76ce9fa8e45402c77382',
 'c509281af915125a436cdadf39fa033b',
 '035ceb268f5009314349c82c4d1c27ce',
 '16199a0571f64744037193f04a625e54',
 'd069e53a7e91116d0643cd0f61d5fde2',
 '96286115c9024ffd3c0497e29ccfef17',
 '10ef838226f2854e1fe800610b6b4142',
 '1f125834a51748004e2e8c4cc4ecabaa',
 'bc440b788946fcae86321ea29c4e46aa',
 'c9580d8dce8b5d083a7d5ca7038ac1fd',
 'c06f04b69609303460fdc3a82c10356f',
 'b5f25546d6298a6d27fb11bc80b5a49a',
 '0007940046105679400000002000000000067805363US',
 '54742f08254bf2e4e870da6bbc15491c',
 '7b05055489

In [55]:
import pandas as pd
import numpy as np

# Step 1: Define the number of users and products
num_users= 1378
num_products = 2220
min_interactions_per_user = 50  # Each user interacts with at least 50 products

# Step 2: Generate unique user IDs and product IDs


# Step 3: Generate a random user-item interaction matrix
np.random.seed(42)  # Set a seed for reproducibility
ratings = np.random.randint(1, 6, size=(num_users, num_products))  # Ratings between 1 and 5

# Create a DataFrame to store the interactions
interaction_matrix = pd.DataFrame(0, index=user_id, columns=ProdID)  # Start with zeros

# Step 4: Assign random ratings to at least 50 products for each user
for user_id in user_id:
    # Select random products for this user (at least 50 products)
    selected_products = np.random.choice(ProdID, min_interactions_per_user, replace=False)
    
    # Assign random ratings to the selected products
    ratings_for_user = np.random.randint(1, 6, size=min_interactions_per_user)  # Ratings between 1 and 5
    interaction_matrix.loc[user_id, selected_products] = ratings_for_user

# Step 5: Display the first few rows of the interaction matrix
print(interaction_matrix.head())


           43844b4d01b62ddf5b329af60afd243a  56182586933bbdea8c572b0af1af513b  \
USER_161                                  0                                 0   
USER_42                                   0                                 0   
USER_1335                                 0                                 0   
USER_34                                   0                                 0   
USER_422                                  0                                 0   

           446f1f57405d2df74e8d89a8dbd458ba  658e17ace9dd43ba0541c590434eb34e  \
USER_161                                  0                                 0   
USER_42                                   0                                 0   
USER_1335                                 0                                 0   
USER_34                                   0                                 0   
USER_422                                  0                                 0   

           ca95004fec7e177

In [56]:
user_item_matrix.to_csv('user_interaction_matrix.csv', index=True)